In [1]:
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
import torch
from PIL import Image
import requests
from rich import print

c:\Users\Khitish.Mohanty.THECONTENTGROUP\.conda\envs\llm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Initialize the processor and model from Hugging Face
processor = LlavaNextProcessor.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf")
model = LlavaNextForConditionalGeneration.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf", torch_dtype=torch.float16, low_cpu_mem_usage=True) 
model.to("cuda:0")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00,  5.29it/s]


AssertionError: Torch not compiled with CUDA enabled

In [3]:
# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

LlavaNextForConditionalGeneration(
  (vision_tower): CLIPVisionModel(
    (vision_model): CLIPVisionTransformer(
      (embeddings): CLIPVisionEmbeddings(
        (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
        (position_embedding): Embedding(577, 1024)
      )
      (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (encoder): CLIPEncoder(
        (layers): ModuleList(
          (0-23): 24 x CLIPEncoderLayer(
            (self_attn): CLIPAttention(
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (mlp): CLIPMLP(
              (activation_fn): Q

In [22]:
# Load an image from your local drive
image_path = ".\images\California_California Government_B61865EB-3E77-4651-B615-7134A17B4BC1.png"  # Replace this with the path to your local image file
image = Image.open(image_path).convert("RGB")

In [21]:
# Example preprocessing (replace with the appropriate preprocessing for LLaVa-NeXT)
def preprocess_image(image):
  # Resize to a specific size (e.g., 224x224)
  image = image.resize((224, 224))
  # Convert to tensor
  image = torch.from_numpy(np.array(image)).float()
  # Normalize (might require specific normalization for LLaVa-NeXT)
  image = image / 255.0
  # Add additional dimensions (batch size, channel order)
  image = image.unsqueeze(0).unsqueeze(0)  # Batch size 1, channel at front
  return image

In [25]:
preprocessed_image = preprocess_image(image)

In [10]:
# Prepare the prompt
text_prompt = "Extract the title from this image."


In [29]:
# Process inputs
inputs = processor(text_prompt, images=image, return_tensors="pt", padding=True)
#inputs = {key: val.to(device) for key, val in inputs.items()}

In [30]:
# Generate text using the model
generated_ids = model.generate(inputs)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


AttributeError: 

In [17]:
# Check input dimensions and types
print("Input IDs shape:", inputs['input_ids'].shape)
print("Attention mask shape:", inputs['attention_mask'].shape)
if 'pixel_values' in inputs:
    print("Pixel values shape:", inputs['pixel_values'].shape)

Input IDs shape: torch.Size([1, 9])
Attention mask shape: torch.Size([1, 9])
Pixel values shape: torch.Size([1, 5, 3, 336, 336])


In [18]:

# Generate text using the model
outputs = model.generate(**inputs, max_length=500)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


ValueError: The input provided to the model are wrong. The number of image tokens is 0 while the number of image given to the model is 1. This prevents correct indexing and breaks batch generation.